In [46]:
import json
import re
import pathlib
import csv

from minicons import scorer
from tqdm import tqdm

In [20]:
# read jsonl
def read_jsonl(path):
    with open(path) as f:
        data = f.readlines()
    data = [json.loads(line) for line in data]
    return data

def find_and_split(sentence, target):
    if target == ".":
        search_query = "\."
    else:
        search_query = target
    search_results = list(re.finditer(search_query, sentence))[-1].span()
    return sentence[:search_results[0]].strip(), target

pipps = read_jsonl('../../data/pipps/materials.jsonl')

In [67]:
# model_name = "kanishka/smolm-autoreg-bpe-babylm-1e-3"
model_name = "gpt2"

lm = scorer.IncrementalLMScorer(model_name)

model_name = (
        model_name.replace("../smolm/models/", "")
        .replace("kanishka/", "")
        .replace("/", "_")
    )

model_name

Using pad_token, but it is not set yet.


'gpt2'

In [61]:
lm.conditional_score(['the keys to the cabinet']*2, ['is', 'are'], reduction=lambda x: -x.mean(0).item())
# lm.conditional_score()

[4.189455032348633, 2.8807172775268555]

In [62]:
for pipp in tqdm(pipps):
    for key in pipp:
        if key not in ['idx', 'embedding', 'preposition']:
            sentence = pipp[key]['sentence']
            target = pipp[key]['target']
            prefix, query = find_and_split(sentence, target)
            if target == ".":
                sep = ""
            else:
                sep = " "
            score = lm.conditional_score(prefix, query, separator=sep, reduction = lambda x: -x.mean(0).item(), base_two=True)
            pipp[key]['score'] = score[0]

100%|██████████| 198/198 [00:33<00:00,  5.90it/s]


In [63]:
def write_to_csv(results, filename):
    with open(filename, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(['idx', 'preposition', 'embedding', 'pipp_filler_gap', 'pp_no_filler_no_gap', 'filler_no_gap', 'no_filler_gap'])
        for result in results:
            writer.writerow([result['idx'], result['preposition'], result['embedding'], result['pipp_filler_gap']['score'], result['pp_no_filler_no_gap']['score'], result['filler_no_gap']['score'], result['no_filler_gap']['score']])

In [64]:
pathlib.Path('../../data/results/pipps/').mkdir(parents=True, exist_ok=True)
write_to_csv(pipps, f'../../data/results/pipps/{model_name}.csv')

In [40]:
lm.token_score("Happy though we were with the idea, we decided to move on.", base_two=True)

[[('<s>', 0.0),
  ('happy', -13.18498706817627),
  ('though', -14.813815116882324),
  ('we', -7.678586959838867),
  ('were', -4.217038154602051),
  ('with', -10.1555814743042),
  ('the', -3.2003817558288574),
  ('idea', -11.002852439880371),
  (',', -5.559576511383057),
  ('we', -4.415450096130371),
  ('decided', -4.9519572257995605),
  ('to', -0.8715379238128662),
  ('move', -5.797823429107666),
  ('on', -2.466400384902954),
  ('.', -2.186781406402588)]]

In [37]:
(-12.507351875305176 -0.00291132228448987)/2

-6.255131598794833

In [65]:
pipps[0]

{'idx': 0,
 'preposition': 'though',
 'embedding': '',
 'pipp_filler_gap': {'sentence': 'Happy though we were with the idea, we decided to move on.',
  'target': 'with',
  'score': 8.902581214904785},
 'pp_no_filler_no_gap': {'sentence': 'Though we were happy with the idea, we decided to move on.',
  'target': 'happy',
  'score': 9.176909446716309},
 'filler_no_gap': {'sentence': 'Happy though we were happy with the idea, we decided to move on.',
  'target': 'happy',
  'score': 10.340938568115234},
 'no_filler_gap': {'sentence': 'Though we were with the idea, we decided to move on.',
  'target': 'with',
  'score': 9.188396453857422}}